# Import libraries

In [ ]:
import requests
import pandas as pd
import pymongo

# Setup MongoDB client and prepare it for storing requests

In [ ]:
mc = pymongo.MongoClient()

In [ ]:
db = mc['games']


In [ ]:
ig_data = db['ig_data']

In [ ]:
ig_data.count_documents({})

# Setup api key from local file in a python environment

In [ ]:
with open('.secrets/igdb-api-key') as f:
    api_key = f.read().strip()


# Prepare variables. url, api_key, and years 2007-2018 in unix format

In [ ]:
url = "https://api-v3.igdb.com/games"
headers = {'user-key': api_key}

dates = [[1167609600, 1199059200], [1199145600,1230681600], [1230768000,1262217600],
        [1262304000,1293753600], [1293840000,1325289600], [1325376000, 1356912000],
        [1356998400, 1388448000], [1388534400, 1419984000], [1420070400, 1451520000],
        [1451606400, 1483142400], [1485820800,1514678400], [1514764800,1546214400]]

Sends a query for top 50 most popular games, and then repeat 4 times for each year.
The resluts would be the top 200 games in each year.

In [ ]:

for year in dates:
    offsets = [0, 50, 100, 150]
    for offset in offsets:
        query = {'fields': f"""*; \
              where themes != (42) & first_release_date > {year[0]} & first_release_date < {year[1]}; \
              sort popularity desc; \
              limit: 50; \
              offset: {offset}"""}
        result = requests.get(url, headers=headers, params=query)
        ig_data.insert_many(result.json())

# Load dataframe and clean it from null resluts. And using only two features, generes and themes.

In [ ]:
df = pd.DataFrame(ig_data.find())

In [ ]:
df.columns

In [ ]:
clean_df = df[['genres', 'themes']]

In [ ]:
clean_df.dropna(inplace=True)

In [ ]:
def change_genre(values):
    new_list = []
    for i in values:
        new_list.append('genre' + str(i))
    return new_list
def change_theme(values):
    new_list = []
    for i in values:
        new_list.append('theme' + str(i))
    return new_list

The default values are stored as lists of numbers, such as [1,3,6,4] each number represents a genre or theme code. It will convert to strings of ['genre1', 'genre4'] for use in one hot encoding later

In [ ]:
clean_df.genres = clean_df.genres.apply(change_genre)
clean_df.themes = clean_df.themes.apply(change_theme)

In [ ]:
clean_df.head()

# one hot encode both features, then concat both dataframes in the final matrix.

In [ ]:
genres_matrix = pd.get_dummies(clean_df.genres.apply(pd.Series), prefix="", prefix_sep="")

In [ ]:
themes_matrix = pd.get_dummies(clean_df.themes.apply(pd.Series), prefix="", prefix_sep="")

In [ ]:
themes_matrix.shape

In [ ]:
genres_matrix.shape

In [ ]:
frames = [themes_matrix, genres_matrix]
matrix_df = pd.concat(frames, axis=1)

In [ ]:
matrix_df